In [1]:
%%html
<style type='text/css'>
.CodeMirror{
font-size: 14px;
</style>
CUDA_LAUNCH_BLOCKING=1

In [2]:
!pip3 install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html


In [3]:
!pip install timm ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [4]:
!pip install seaborn
!pip install pydub
!pip install ffmpeg

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [5]:
!pip install git+https://github.com/KinWaiCheuk/nnAudio.git#subdirectory=Installation

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/KinWaiCheuk/nnAudio.git to /tmp/pip-req-build-36bqq8gb
  Running command git clone -q https://github.com/KinWaiCheuk/nnAudio.git /tmp/pip-req-build-36bqq8gb
  Resolved https://github.com/KinWaiCheuk/nnAudio.git to commit 744fab12497a5316153978de2e97422c9c7389e0


In [6]:
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

env: CUBLAS_WORKSPACE_CONFIG=:4096:8


In [7]:
# humbug main imports

import os
import seaborn as sns
import pandas as pd
import sys
sys.path.insert(0, os.path.abspath('../lib'))
import config
from evaluate import get_results 
import numpy as np

# Troubleshooting and visualisation
import IPython.display as ipd

In [8]:
# humbug lib imports
from sklearn.metrics import accuracy_score
from PyTorch import config_pytorch
from datetime import datetime
import math
import pickle

from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch
import torch.optim as optim
import numpy as np
from sklearn.metrics import accuracy_score
from datetime import datetime
import os
import time

import matplotlib
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_curve, plot_precision_recall_curve
from sklearn.metrics import average_precision_score
import sys
#import seaborn as sns

from tqdm.notebook import tqdm

In [9]:
# additional pytorch tools
import random
import torchaudio
import torchaudio.transforms as T
import torchvision.transforms as VT
from torch.cuda.amp import autocast, GradScaler
from timm.scheduler.cosine_lr import CosineLRScheduler
import timm
import timm.optim
from timm.loss import LabelSmoothingCrossEntropy
from timm.utils import NativeScaler
from timm.models import model_parameters
from glob import glob

In [10]:
## nnAudio
from nnAudio import features
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader , WeightedRandomSampler

In [11]:
#check the min length based on config params
min_length = (config.win_size * config.n_hop) / config.rate
min_length

1.92

In [12]:
# use audio clips shorter than min length ?
USE_SHORT_AUDIO = True

##  Class defintion "Others"
This should be in accordance with the pre-trained model.
Class "others" is a ctach-all for all mosq species tht are not indicated explicitly in the list below

In [13]:
classes = ['an arabiensis','culex pipiens complex', 'ae aegypti','an funestus ss','an squamosus',
               'an coustani','ma uniformis','ma africanus','others']






## Step 2: Class Definitions

In [14]:
config_pytorch.batch_size = 40
config_pytorch.epochs = 75

#https://discuss.pytorch.org/t/how-to-handle-imbalanced-classes/11264


In [15]:
class MozDatasetMED(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, audio_df, data_dir, min_length,  transform=None):
        """
        Args:
            audio_df (DataFrame): from get_offsets_df function 
            data_dir (string): Directory with all the wavs.
            cache (dict): Empty dictionary used as cache
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.audio_df = audio_df
        self.data_dir = data_dir
        self.min_length = min_length
        self.transform = transform
        self.cache = None

    def __len__(self):
        return len(self.audio_df)
    
    def _get_sample_(self, path, resample=None):
        if self.cache is not None and path in self.cache:
            return self.cache[path], resample
        else:
            effects = [
                ["remix", "1"]
            ]
            if resample:
                effects.extend([
                  ["bandpass", f"{config.lower_bnd}",f"{config.upper_bnd}"],
                  ["rate", f'{resample}'],
                  ['gain', '-n'],
#                 ["highpass", "600"],
                ])
            waveform, rate = torchaudio.sox_effects.apply_effects_file(path, effects=effects)
            f = waveform[0]
            mu = torch.std_mean(f)[1]
            st = torch.std_mean(f)[0]
            #return waveform, rate, waveform
            f_out = torch.clamp(f, min=mu-st*3, max=mu+st*3).unsqueeze(0)
            if self.cache is not None:
                self.cache[path] = f_out
        return f_out, rate

    def __getitem__(self, idx):
        real_idx = idx % len(self.audio_df)
        x, _ = self._get_sample_(os.path.join(self.data_dir,f"{self.audio_df.loc[real_idx]['id']}.wav"), resample=config.rate)
                                   
        offset = self.audio_df.loc[real_idx]['offset']
        if self.audio_df.loc[real_idx]['sound_type'] == 'mosquito' :
            return x[:,offset:int(offset+config.rate*self.min_length)], 1
        else:
            return x[:,offset:int(offset+config.rate*self.min_length)], 0
    


In [16]:
class MozTestDatasetMSC(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, audio_df, data_dir, min_length, transform=None):
        """
        Args:
            audio_df (DataFrame): from get_offsets_df function 
            data_dir (string): Directory with all the wavs.
            cache (dict): Empty dictionary used as cache
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.audio_df = audio_df
        self.data_dir = data_dir
        self.min_length = min_length
        self.transform = transform
        self.cache = None

    def __len__(self):
        return len(self.audio_df)
    
    def _get_sample_(self, path, resample=None):
        if self.cache is not None and path in self.cache:
            return self.cache[path], resample
        else:
            effects = [
                ["remix", "1"]
            ]
            if resample:
                effects.extend([
                  ["rate", f'{resample}'],
                  ['gain', '-n'],
                  ["highpass", "200"],
                ])
            waveform, rate = torchaudio.sox_effects.apply_effects_file(path, effects=effects)
            #print("wave shape = " + str(waveform.shape))
            if waveform.shape[1] < config.rate*self.min_length:
                left_pad_amt = int((config.rate*min_length-waveform.shape[1])//2)
                #print("left_pad_amt = " + str(left_pad_amt))
                left_pad = torch.zeros(1,left_pad_amt) #+ (0.1**0.5)*torch.randn(1, left_pad_amt)
                right_pad_amt = int(config.rate*min_length-waveform.shape[1]-left_pad_amt)
                right_pad = torch.zeros(1,right_pad_amt)# + (0.1**0.5)*torch.randn(1, right_pad_amt)
                f = torch.cat([left_pad,waveform,right_pad],dim=1)[0]
                #print("f shape = " + str(f.shape))
                f = ((0.0001**0.5)*torch.randn(1, int(config.rate*min_length))+f)[0]
#                 r = math.ceil((config.rate*self.min_length)/waveform.shape[1])
#                 f = torch.cat([waveform for _ in range(r)],dim=1)[:,:int(config.rate*self.min_length)][0]
            else:    
                f = waveform[0]
            mu = torch.std_mean(f)[1]
            st = torch.std_mean(f)[0]
            #return waveform, rate, waveform
            f_out = torch.clamp(f, min=mu-st*3, max=mu+st*3).unsqueeze(0)
            f_out = f.unsqueeze(0)
            if self.cache is not None:
                self.cache[path] = f_out
        return f_out, rate

    def __getitem__(self, idx):
        real_idx = idx % len(self.audio_df)
        #print("real_idx = " + str(real_idx))
        #print("INSIDE TEST ....idx passed is --" + str(idx))
        real_idx = idx % len(self.audio_df)
        #print("INSIDE TEST REAL idx  is --" + str(real_idx))
        #x, _ = self._get_sample_(os.path.join(self.data_dir,f"{self.audio_df.loc[real_idx]['id']}.wav"), resample=config.rate)
        x, _ = self._get_sample_(os.path.join(self.data_dir,f"{self.audio_df.loc[real_idx]['id']}.wav"), resample=config.rate)
        
        
        
        offset = self.audio_df.loc[real_idx]['offset']
        #return (x[:,offset:int(offset+config.rate*self.min_length)], self.audio_df.loc[real_idx]['species'] )
        return (x[:,offset:int(offset+config.rate*self.min_length)], self.audio_df.loc[real_idx]['specie_ind'] )


In [17]:
class ModelMED_new(nn.Module):
    def __init__(self, model_name, image_size):
        super().__init__()
        # num_classes=0 removes the pretrained head
        self.backbone = timm.create_model(model_name,
                        pretrained=True, num_classes=2, in_chans=1, 
                        drop_path_rate=0.1, global_pool='avgmax',
                        drop_rate=0.1)
        #####  This section is model specific
        #### It freezes some fo the layers by name
        #### you'll have to inspect the model to see the names
        for name, param in self.backbone.named_parameters():
            if param.requires_grad and 'head' not in name \
                and not name.startswith('norm') \
                and 'stages.3' not in name and 'layers.3' not in name \
                and 'blocks.26' not in name and 'blocks.26' not in name \
                and 'blocks.24' not in name and 'blocks.25' not in name \
                and 'blocks.22' not in name and 'blocks.23' not in name \
                and 'blocks.20' not in name and 'blocks.21' not in name \
                and 'blocks.22' not in name and 'blocks.23' not in name \
                and 'blocks.19' not in name and 'blocks.18' not in name \
                and 'blocks.17' not in name and 'blocks.5.' not in name:
                param.requires_grad = False
        #### end layer freezing
        self.spec_layer = features.STFT(n_fft=config.NFFT, freq_bins=None, hop_length=config.n_hop,
                              window='hann', freq_scale='log2', center=True, pad_mode='reflect',
                          fmin=300, fmax=3000, sr=config.rate, output_format="Magnitude", trainable=True)
#         self.spec_layer = features.MelSpectrogram(n_fft=config.NFFT, n_mels=128, hop_length=config.n_hop,
#                                 window='hann',  center=True, pad_mode='reflect',
#                           sr=config.rate,  trainable_mel=True, trainable_STFT=True)
#         self.out = nn.Linear(self.backbone.num_features, 1)
        self.sizer = VT.Resize((image_size,image_size))
        self.timeMasking = T.TimeMasking(time_mask_param=30, iid_masks=True)
        self.freqMasking = T.FrequencyMasking(freq_mask_param=20, iid_masks=True)
        self.norm_layer = Normalization(mode='framewise')
        self.pcen_layer = PCENTransform(eps=1e-6, s=0.025, alpha=0.6, delta=0.1, r=0.2, trainable=True)
        
    def forward(self, x):
        # first compute spectrogram
        spec = self.spec_layer(x)  # (B, F, T)
        # normalize
#         spec = spec.transpose(1,2) # (B, T, F)
        spec = self.pcen_layer(spec)
        spec = self.norm_layer(spec)
        
#         if self.training:
#             spec = self.timeMasking(spec)
#             spec = self.freqMasking(spec)

        # then size for CNN model
        # and create a channel
        spec = self.sizer(spec)
        x = spec.unsqueeze(1)
        # then repeat channels
        pred = self.backbone(x)
        
#        pred = self.out(x)
        
        output = {"prediction": pred,
                  "spectrogram": spec}
        return output

In [18]:
class ModelMED(nn.Module):
    def __init__(self, model_name, image_size):
        super().__init__()
        # num_classes=0 removes the pretrained head
        self.backbone = timm.create_model(model_name,
                        pretrained=True, num_classes=0, in_chans=1, 
                        drop_path_rate=0.05, global_pool='max',
                        drop_rate=0.05)
        #####  This section is model specific
        #### It freezes some fo the layers by name
        #### you'll have to inspect the model to see the names
        for name, param in self.backbone.named_parameters():
            if param.requires_grad and 'head' not in name \
                and not name.startswith('norm') \
                and 'stages.3' not in name \
                and 'blocks.26' not in name and 'blocks.26' not in name \
                and 'blocks.24' not in name and 'blocks.25' not in name \
                and 'blocks.22' not in name and 'blocks.23' not in name \
                and 'blocks.20' not in name and 'blocks.21' not in name \
                and 'blocks.22' not in name and 'blocks.23' not in name \
                and 'blocks.19' not in name and 'blocks.18' not in name \
                and 'blocks.17' not in name:
                param.requires_grad = False
        #### end layer freezing
        self.spec_layer = features.STFT(n_fft=config.NFFT, freq_bins=None, hop_length=config.n_hop,
                              window='hann', freq_scale='linear', center=True, pad_mode='reflect',
                          fmin=400, fmax=2000, sr=config.rate, output_format="Magnitude", trainable=True)
        
        self.out = nn.Linear(self.backbone.num_features, 1)
        self.sizer = VT.Resize((image_size,image_size))
        self.timeMasking = T.TimeMasking(time_mask_param=int(config.win_size*0.4), iid_masks=True)
        self.freqMasking = T.FrequencyMasking(freq_mask_param=int((config.NFFT//4)*0.15), iid_masks=True)
        self.norm_layer = Normalization(mode='framewise')
        self.pcen_layer = PCENTransform(eps=1e-6, s=0.025, alpha=0.6, delta=0.1, r=0.2, trainable=True)
        
    def forward(self, x):
        # first compute spectrogram
        spec = self.spec_layer(x)  # (B, F, T)
        #print("self.spec_layer shape = " + str(spec.shape))
#         spec = spec.transpose(1,2) # (B, T, F)
        spec = self.pcen_layer(spec)
        #print("self.pcen_layer shape = " + str(spec.shape))
        spec = self.norm_layer(spec)
        #print("self.norm_layer shape = " + str(spec.shape))
        
#         if self.training:
        spec = self.timeMasking(spec)
        #print("self.timemasking shape = " + str(spec.shape))
        spec = self.freqMasking(spec)
        #print("self.freqmasking shape = " + str(spec.shape))

        # then size for CNN model
        # and create a channel
        spec = self.sizer(spec)
        x = spec.unsqueeze(1)
        # then repeat channels
        x = self.backbone(x)
        #print("tensor that goes in the model shape = " +str(x.shape))
        
        pred = self.out(x)
        #print("Pred shape = " +str(pred.shape))
        
        output = {"prediction": pred,
                  "spectrogram": spec}
        return output

In [19]:
class ModelMSC(nn.Module):
    def __init__(self, model_name, image_size):
        super().__init__()
        # NOTE - hard coded num classes
        self.backbone =  timm.create_model(model_name,
                        pretrained=True, num_classes=9, in_chans=1, 
                        drop_path_rate=0.5, global_pool='max',
                        drop_rate=0.5)
       
        #####  This section is modelspecific
        #### It freezes some fo the layers by name
        #### you'll have to inspect the model to see the names
        for name, param in self.backbone.named_parameters():
            if param.requires_grad and 'head' not in name \
                and not name.startswith('norm') \
                and 'stages.3' not in name and 'layers.3' not in name \
                and 'blocks.26' not in name and 'blocks.26' not in name \
                and 'blocks.24' not in name and 'blocks.25' not in name \
                and 'blocks.22' not in name and 'blocks.23' not in name \
                and 'blocks.20' not in name and 'blocks.21' not in name \
                and 'blocks.22' not in name and 'blocks.23' not in name \
                and 'blocks.19' not in name and 'blocks.18' not in name \
                and 'blocks.17' not in name and 'blocks.5.' not in name:
                param.requires_grad = False
        #### end layer freezing
        self.spec_layer = features.STFT(n_fft=config.NFFT, freq_bins=None, hop_length=config.n_hop,
                              window='hann', freq_scale='log2', center=True, pad_mode='reflect',
                          fmin=300, fmax=3000, sr=config.rate, output_format="Magnitude", trainable=True)
#         self.spec_layer = features.MelSpectrogram(n_fft=config.NFFT, n_mels=128, hop_length=config.n_hop,
#                                 window='hann',  center=True, pad_mode='reflect',
#                           sr=config.rate,  trainable_mel=True, trainable_STFT=True)
#         self.out = nn.Linear(self.backbone.num_features, 1)
        self.sizer = VT.Resize((image_size,image_size))
        self.timeMasking = T.TimeMasking(time_mask_param=30, iid_masks=True)
        self.freqMasking = T.FrequencyMasking(freq_mask_param=20, iid_masks=True)
        self.norm_layer = Normalization(mode='framewise')
        self.pcen_layer = PCENTransform(eps=1e-6, s=0.025, alpha=0.6, delta=0.1, r=0.2, trainable=True)
        
    def forward(self, x):
        # first compute spectrogram
        spec = self.spec_layer(x)  # (B, F, T)
        # normalize
#         spec = spec.transpose(1,2) # (B, T, F)
        spec = self.pcen_layer(spec)
        spec = self.norm_layer(spec)
        
#         if self.training:
        spec = self.timeMasking(spec)
        spec = self.freqMasking(spec)

        # then size for CNN model
        # and create a channel
        spec = self.sizer(spec)
        x = spec.unsqueeze(1)
        # then repeat channels
        pred = self.backbone(x)
        
#        pred = self.out(x)
        
        output = {"prediction": pred,
                  "spectrogram": spec}
        return output

In [20]:
def pcen(x, eps=1e-6, s=0.025, alpha=0.98, delta=2, r=0.5, training=False):
    frames = x.split(1, -2)
    m_frames = []
    last_state = None
    for frame in frames:
        if last_state is None:
            last_state = s * frame
            m_frames.append(last_state)
            continue
        if training:
            m_frame = ((1 - s) * last_state).add_(s * frame)
        else:
            m_frame = (1 - s) * last_state + s * frame
        last_state = m_frame
        m_frames.append(m_frame)
    M = torch.cat(m_frames, 1)
    if training:
        pcen_ = (x / (M + eps).pow(alpha) + delta).pow(r) - delta ** r
    else:
        pcen_ = x.div_(M.add_(eps).pow_(alpha)).add_(delta).pow_(r).sub_(delta ** r)
    return pcen_


class PCENTransform(nn.Module):

    def __init__(self, eps=1e-6, s=0.025, alpha=0.98, delta=2, r=0.5, trainable=True):
        super().__init__()
        if trainable:
            self.log_s = nn.Parameter(torch.log(torch.Tensor([s])))
            self.log_alpha = nn.Parameter(torch.log(torch.Tensor([alpha])))
            self.log_delta = nn.Parameter(torch.log(torch.Tensor([delta])))
            self.log_r = nn.Parameter(torch.log(torch.Tensor([r])))
        else:
            self.s = s
            self.alpha = alpha
            self.delta = delta
            self.r = r
        self.eps = eps
        self.trainable = trainable

    def forward(self, x):
#         x = x.permute((0,2,1)).squeeze(dim=1)
        if self.trainable:
            x = pcen(x, self.eps, torch.exp(self.log_s), torch.exp(self.log_alpha), torch.exp(self.log_delta), torch.exp(self.log_r), self.training and self.trainable)
        else:
            x = pcen(x, self.eps, self.s, self.alpha, self.delta, self.r, self.training and self.trainable)
#         x = x.unsqueeze(dim=1).permute((0,1,3,2))
        return x

In [21]:
class Normalization():
    """This class is for normalizing the spectrograms batch by batch. The normalization used is min-max, two modes 'framewise' and 'imagewise' can be selected. In this paper, we found that 'imagewise' normalization works better than 'framewise'"""
    def __init__(self, mode='framewise'):
        if mode == 'framewise':
            def normalize(x):
                size = x.shape
                x_max = x.max(1, keepdim=True)[0] # Finding max values for each frame
                x_min = x.min(1, keepdim=True)[0]  
                output = (x-x_min)/(x_max-x_min) # If there is a column with all zero, nan will occur
                output[torch.isnan(output)]=0 # Making nan to 0
                return output
        elif mode == 'imagewise':
            def normalize(x):
                size = x.shape
                x_max = x.reshape(size[0], size[1]*size[2]).max(1, keepdim=True)[0]
                x_min = x.reshape(size[0], size[1]*size[2]).min(1, keepdim=True)[0]
                x_max = x_max.unsqueeze(1) # Make it broadcastable
                x_min = x_min.unsqueeze(1) # Make it broadcastable 
                return (x-x_min)/(x_max-x_min)
        else:
            print(f'please choose the correct mode')
        self.normalize = normalize

    def __call__(self, x):
        return self.normalize(x)

## Inference

In [22]:

# pre - trained models
#MED_file = "../outputs/models/pytorch/model_e9_2022_07_30_23_20_04.pth"
MED_file = "../outputs/models/pytorch/model_e18_2022_08_20_16_54_26.pth"
MSC_file = "../outputs/models/pytorch/model_e3_2022_07_30_20_36_28.pth"

In [23]:
model_MED = ModelMED_new('convnext_small',224)
model_MSC = ModelMSC('convnext_small',224)


sampling rate = 8000. Please make sure the sampling rate is correct in order toget a valid freq range
STFT kernels created, time used = 0.0881 seconds
sampling rate = 8000. Please make sure the sampling rate is correct in order toget a valid freq range
STFT kernels created, time used = 0.0797 seconds


In [24]:
model_MED.load_state_dict(torch.load(MED_file))
model_MSC.load_state_dict(torch.load(MSC_file))

model_MED = model_MED.eval()
model_MSC = model_MSC.eval()

In [25]:
# temp_x = torch.rand(1,15360)
# y_pred = model_MSC(temp_x)['prediction']
# print(y_pred)



In [26]:
# This function pads a file with 0s to make it a 1.92 sec file
def pad_zero(x_temp,rate = config.rate, min_length = config.min_duration ):
    #print("inside padding zero...")
    left_pad_amt = int((rate*min_length-x_temp.shape[1])//2)
    #print("left_pad_amt = " + str(left_pad_amt))
    left_pad = torch.zeros(1,left_pad_amt) #+ (0.1**0.5)*torch.randn(1, left_pad_amt)
    right_pad_amt = int(rate*min_length-x_temp.shape[1]-left_pad_amt)
    right_pad = torch.zeros(1,right_pad_amt)# + (0.1**0.5)*torch.randn(1, right_pad_amt)
    f = torch.cat([left_pad,x_temp,right_pad],dim=1)[0]
    #print("returning a tensor of shape = " + str(f.shape))
    return(f)
    

In [27]:
#x = torch.randn(1,int(config.rate*min_length))
#y_pred = model_MED(x)['prediction']
# print(y_pred)

In [28]:
#model_MED = model_MED_init.load_state_dict(checkpoint_MED_temp['model_state_dict'])

In [29]:
# This function plots the entire specie prediction in one bar plot
def plot_specie(df):
    from matplotlib import pyplot as plt
    import seaborn as sns
    dfm = df.melt(id_vars='offset')
    #start = round((ind *min_duration),2)
    #end = round((start + min_duration),2)
    #title_str = "Plot for the timestamp = " + str(start) + " " + "to " + str(end) + " seconds"
    sns.set_context("notebook")
    #a = np.array(y_hat.detach().cpu())
    #dict_y_hat = dict.fromkeys(classes)
    #for i in range(0,a.shape[1]):
    #    dict_y_hat[classes[i]] = round(a[0][i],2)
    #df_y_hat_prob = pd.DataFrame(dict_y_hat ,index = [0])
    #df_y_hat_prob['offset'] = ind
    
    #print(df_y_hat_prob)
    
    
    plt.figure(figsize=(20,10))
    g = sns.barplot(data = dfm,x='variable',y = 'value', hue='offset'  )
    #g = sns.barplot(data = df,orient = "v" , x = "offset" , y  = df.columns[1:8])
   # g.set_title(title_str)
    plt.xticks(rotation=90);
    g.set(xlabel="Specie",ylabel="Probability");
    g.bar_label(g.containers[0])
    

In [30]:
def get_specie(x ,model_MSC,classes,i,y_hat_mosq,y_hat_noise,threshold, zero = False):
    
    if zero == True:
        #print("inside get_specie . Zero val = " + str(zero))
        zero_array = np.zeros((1,9),dtype = float)
        dict_y_hat = dict.fromkeys(classes)
        for j in range(0,zero_array.shape[1]):
            dict_y_hat[classes[j]] = round(zero_array[0][j],2)
        df_y_hat_prob = pd.DataFrame(dict_y_hat ,index = [i])
        df_y_hat_prob['offset'] = i
        df_y_hat_prob['threshold'] = threshold
        df_y_hat_prob['MED_mozz'] = y_hat_mosq
        df_y_hat_prob['MED_noise'] = y_hat_noise
        
        
        df_tup = df_y_hat_prob
        #print("df_y_hat_prob = ")
        #print(df_y_hat_prob)
        return (df_tup)
         
    else:
        with torch.no_grad():
            #print("inside get_specie  ...ZERO = FALSE")
            device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
            softmax = nn.Softmax(dim=1)
            #y_preds_all = torch.zeros([n_samples, len(test_loader.dataset), 9])
            model_MSC.eval() # Important to not leak info from batch norm layers and cause other issues
            model_MSC.to(device)
            x = x.to(device)
            y_pred = model_MSC(x)['prediction']
            #print("Prediction score = " + str(y_pred))
            #all_y.append(y.cpu().detach())
            y_hat = softmax(y_pred)
            #print("y_hat = " + str(y_hat))
            a = np.array(y_hat.detach().cpu())
            dict_y_hat = dict.fromkeys(classes)
            for j in range(0,a.shape[1]):
                dict_y_hat[classes[j]] = round(a[0][j],2)
            df_y_hat_prob = pd.DataFrame(dict_y_hat ,index = [i])
            df_y_hat_prob['offset'] = i
            df_y_hat_prob['threshold'] = threshold
            df_y_hat_prob['MED_mozz'] = y_hat_mosq
            df_y_hat_prob['MED_noise'] = y_hat_noise
            
            df_tup = df_y_hat_prob
            #print("df_y_hat_prob = ")
            #print(df_y_hat_prob)
        return df_tup
        #plot_specie(y_hat , ind ,min_duration = config.min_duration)
        
      


In [39]:
def aac_to_wav(aac_file , file_name , rate = config.rate):
    path = "../data/audio/"
    wav_name = "../data/audio/" + file_name + ".wav"
    from os.path import exists
    file_exists = exists(aac_file)
    print("file_exists = " +str(file_exists))
    from pydub import AudioSegment
    track = AudioSegment.from_file(aac_file,  format= 'aac')
    file_handle = track.export(wav_name, format='wav')
    return file_handle


inp = "r2022-07-26_00.00.00.410__v6.aac_u2022-07-26_06.00.24.521642.aac"
split_tup = os.path.splitext(inp)
print(split_tup)
if split_tup[1] != ".wav" :
    print("not wav")
    inp_new = "../data/audio/"+ inp
    print("inp_new = " +str(inp_new))
    hand = aac_to_wav(inp_new , split_tup[0])
    #wav = "../data/audio/" + split_tup[0]+".wav"
  
    #print("wav = " + str(wav))
    inf_tensor,inp_samp_rate = torchaudio.load(hand)
    print(inf_tensor.shape)


In [45]:
# Main function that reads a file for inference and predicts
def infer_model(model_MED, model_MSC ,file , classes ,rate = config.rate ):
    threshold = .5
    softmax = nn.Softmax(dim=1)
    print("inside infer_model ....file = " + str(file))
    split_tup = os.path.splitext(file)
    if split_tup[1] != ".wav" :
        inp_new = "../data/audio/"+ inp
        inf_file = aac_to_wav(inp_new , split_tup[0])
        inf_tensor,inp_samp_rate = torchaudio.load(inf_file)
        if inp_samp_rate != rate:
            import torchaudio.transforms as T
            resampler = T.Resample(inp_samp_rate, rate, dtype=inf_tensor.dtype)
            inf_tensor = resampler(inf_tensor)
    
    else:
        inf_file = "../data/audio/" + file
        inf_tensor,inp_samp_rate = torchaudio.load(inf_file)
        if inp_samp_rate != rate:
            import torchaudio.transforms as T
            resampler = T.Resample(inp_samp_rate, rate, dtype=inf_tensor.dtype)
            inf_tensor = resampler(inf_tensor)
        
        
    
    
        
    #print("Inference tensor shape = " + str(inf_tensor.shape))
    inf_tensor.shape[1]
    num_inf_win = math.ceil(inf_tensor.shape[1]/(config.rate*min_length))
    print("number of inference windows = " + str(num_inf_win))
    stride = int(config.rate*min_length)
    #print("stride ->" +str(stride))
    # initializing a set up to store predicted values
    zero_array = np.zeros((1,9),dtype = float)
    dict_y_hat = dict.fromkeys(classes)
    df_y_hat = pd.DataFrame(dict_y_hat ,index = [0])
    df_y_hat['offset'] = None
    df_y_hat['threshold'] = None
    df_y_hat['MED_mozz'] = None
    df_y_hat['MED_noise'] = None
    list_of_dfs = []
    #print("df_y_hat = ")
    #print(df_y_hat)
      
    for i in range(0,num_inf_win):
        x_temp = inf_tensor[:,i*stride:(i+1)*stride]
        print("Frame = " + str(i))
        #print("x shape = " + str(x_temp.shape))
        if x_temp.shape[1] < stride :
            print("need to pad")
            x_pad = pad_zero(x_temp)
            x_pad = x_pad.unsqueeze(dim = 0)
            #print("x_pad shape = " + str(x_pad.shape))
            y_pred = model_MED(x_pad)['prediction']
            y_hat = softmax(y_pred)
            #y_hat = torch.tensor([[0.9435, 0.0565]])
            print("y_hat = " + str(y_hat))
            y_hat_noise = (y_hat.detach().cpu()[0][0]).numpy()
            y_hat_mosq = (y_hat.detach().cpu()[0][1]).numpy()
            #print("y_hat_noise  = " + str(y_hat_noise))
            #print("y_hat_mosq  = " + str(y_hat_mosq))
            #print("y_pred = " + str(y_pred))
            #print("y_pred = " + str(y_pred) + "Post detach device = " + str(y_pred.device))
            #print("prob_MED  = " + str(softmax(y_pred)))
            if y_hat_mosq < threshold :
                y_hat = 0
            else:
                y_hat = 1
                
            #print("y_hat = " + str(y_hat))
            if y_hat == 1 :
                print(" ***********Mosq Detected*********** ")
                print("mosq detected from " + str((i)*1.92) +" seconds " + " to " + str((i+1)*1.92) + " seconds ")
                #
                df_temp = get_specie(x_temp ,model_MSC,classes,i,y_hat_mosq,y_hat_noise,threshold,zero = False)
                
            else:
                print("no mosq detected from " + str((i)*1.92) +" seconds " + " to " + str((i*1.92)) + " seconds ")
                df_temp = get_specie(x_temp ,model_MSC,classes,i,y_hat_mosq,y_hat_noise,threshold, zero = True)
            
            df_y_hat = pd.concat([df_y_hat , df_temp] )
        
        else:
            #longer file
            #print("inside long file..")
            y_pred = model_MED(x_temp)['prediction']
            #print("y_pred = " + str(y_pred) + "Pre detach device = " + str(y_pred.device))
            y_pred = y_pred.detach().cpu()
            #y_hat = softmax(y_pred)
            #print("y_pred = " + str(y_pred) + " Post Detach device = " + str(y_pred.device))
            
            y_hat = softmax(y_pred)
            #print("y_hat = " + str(y_hat))
            y_hat_noise = (y_hat.detach().cpu()[0][0]).numpy()
            y_hat_mosq = (y_hat.detach().cpu()[0][1]).numpy()
            #print("y_hat_noise  = " + str(y_hat_noise))
            #print("y_hat_mosq  = " + str(y_hat_mosq))
            
            if y_hat_mosq < threshold :
                y_hat = 0
            else:
                y_hat = 1            
                 
            if y_hat == 1 :
                print(" ***********Mosq Detected*********** ")
                print("mosq detected from " + str((i)*1.92) +" seconds " + " to " + str((i+1)*1.92) + " seconds ")
                df_tup = get_specie(x_temp ,model_MSC,classes,i,y_hat_mosq,y_hat_noise,threshold, zero = False)
                list_of_dfs.append(df_tup)
                #df_y_hat = pd.concat([df_y_hat , df_temp ])
                
                
                
            else:#y_hat == 0
                print(" NO mosq detected from " + str((i)*1.92) +" seconds " + " to " + str((i+1)*1.92) + " seconds ")
                df_tup = get_specie(x_temp ,model_MSC,classes,i,y_hat_mosq,y_hat_noise,threshold, zero = True)
                list_of_dfs.append(df_tup)
                #df_y_hat = pd.concat([df_y_hat , df_temp ])
            
        list_of_dfs.append(df_y_hat)
        df_final = pd.concat(list_of_dfs, ignore_index=True)
            
        
    return df_final
        
        

## User Input for inference

In [46]:
def write_file(df , out_dir,i,classes):
    print("inside writing a text file")
    #print(df)
    #open a file
    file_name = out_dir+"/"+str(i)+".txt"
    with open(file_name, 'w') as f:
        for idx, row in df.iterrows():
            #print("idx = "  + str(idx))
            start_time = int(row['offset'])*1.92
            end_time = int(row['offset']+1)*1.92
            specie_cols = pd.to_numeric(row[0:9])
            threshold = row['threshold']
            if row['MED_mozz'] < threshold:
                noise_prob = row['MED_noise']
                f.write(str(start_time) + "\t" +  str(end_time) + "\t" + "NO MOSQ DETECTED"  + '_P_Noise'+ str(round(noise_prob,3)) + "\n")
                continue
                
            print(specie_cols)
            max_prob = specie_cols.max()
            likely_specie = specie_cols.idxmax()
            #print('Output of argmax->',likely_specie)
            #print("max_prob = " + str(max_prob))
            #print("len(specie_cols) = " + str(len(specie_cols)))
            #print("index of col with max val is = " + str(maxValueIndex))
            #col_name = row.columns[maxValueIndex]
            #prob = row[col_name]
            #print("most likely class is = " + str(prob))
            mozz_prob = row['MED_mozz']
            f.write(str(start_time) + "\t" +  str(end_time) + "\t" + "Most_likely->" + str(likely_specie) + '_P'+ str(round(max_prob,3)) + "_P:MED_MOZZ:" + str(round(mozz_prob,3))  + "\n")
            for i in range(0,len(specie_cols)):
                #print("i = " + str(i) )
                specie_prob = specie_cols[i]
                #print("specie_prob = " + str(round(specie_prob,3)))
                #curr_specie = specie_cols[specie_cols == specie_prob].index[0]
                curr_specie = classes[i]
                #print("curr_specie = " + str(curr_specie))
                f.write(str(start_time) + "\t" +  str(end_time) + "\t" + str(curr_specie) + '_P'+ str(round(specie_prob,3)) + "\n")
                
            
            
    

In [47]:
def concat_frames(df,wav_id,threshold = .5 ,every_mozz_frame = False , rate = config.rate):
    """This function takes the output of MED and the original wav file as input and then concatenates
    successive 1.92 secs frame
    df = output of MED and MSC
    inf_file = the raw file which was provided as input to the original inference
    """
    inf_file = "../data/audio/" + str(wav_id) +".wav"
    inf_tensor,inp_samp_rate = torchaudio.load(inf_file)
    import torchaudio.transforms as T
    if inp_samp_rate != rate:
        resampler = T.Resample(inp_samp_rate, rate, dtype=inf_tensor.dtype)
        inf_tensor = resampler(inf_tensor)
        
    print("concat_frames ....Inference tensor shape = " + str(inf_tensor.shape))
        
    start_ind = 0
    next_ind = start_ind + 1
    offset_to_concat = []
    eolist_flag = False
    # loop that travers the length of the df.
    print("len df = " + str(len(df)))
    while(start_ind < len(df)-1):
        if (next_ind == len(df)):
            #print("reached at the end ...")
            eolist_flag = True
            break
        else:
            #get the prediction at the current index
            row_curr = (df[df['offset'] == start_ind])
            row_next = (df[df['offset'] == next_ind])
            # get the prediction of the model at the current index
            if (row_curr['MED_mozz'] > threshold).bool():
                #print("prediction at" + str(start_ind) + " is mozz ")
                curr_pred = "mozz"
                #when current pred is mozz , check the next frame
                if (row_next['MED_mozz'] > threshold).bool():
                    #print("offset " + str(start_ind) + " and offset " + str(next_ind) + "are mozz" )
                    offset_to_concat.append(start_ind)
                    offset_to_concat.append(next_ind)
                    #print("updating the list for concatanation")
                    #print("post concate list = " + str(offset_to_concat))
                    ###
                    #temp_tensor = logic for concatenate
                    ###
                    start_ind = next_ind
                    next_ind = start_ind + 1
                    #print("post concatenattion , incrementing the next index .next_ind =  " + str(next_ind))
                    if (next_ind == len(df)):
                        print("reached at the end ...")
                        eolist_flag = True
                        break
                
                else:
                    #print("offset " + str(start_ind) +  " is a mozz " + " but offset " + str(next_ind) + "is noise" )
                    start_ind = next_ind
                    next_ind = start_ind + 1
                    #print("start_ind = " + str(start_ind))
                    #print("next_ind = " + str(next_ind))
                
            else:
                curr_pred = "noise"
                #print("Current offset " + str(start_ind) +  " is noise  " )
                #print("incrementing indeices.....")
                #check if the list to concat is empty.if it is empty then update the indices.
                # This indicates that there is nothing to concatenate yet. All files are Noise
                if (len(offset_to_concat) == 0):
                    #print("Curr prediction is noise")
                    start_ind = next_ind
                    next_ind = start_ind + 1
                    #print("start_ind = " + str(start_ind))
                    #print("next_ind = " + str(next_ind))
                    
                else:
                    # else you write the file to the location with specific details , updte the indices and # empty the list
                    #This condition indicates that there was a positive detection earlier and concatenation should happen
                    #print("concatinating...")
                    min_offset = min(offset_to_concat)
                    max_offset = max(offset_to_concat)
                    #print("concatenating from " + str(min_offset*1.92) + " to " + str(max_offset*1.92) )
                    tensor_to_wav(wav_id,inf_tensor , min_offset , max_offset)
                    #print("now emptying the offset list...")
                    offset_to_concat.clear()
                    start_ind = next_ind
                    next_ind = start_ind + 1
                    #print("start_ind = " + str(start_ind))
                    #print("next_ind = " + str(next_ind))
                
    if eolist_flag :
        #print("inside eol list flag")
        #print("concatinating...")
        min_offset = min(offset_to_concat)
        max_offset = max(offset_to_concat)
        #print("concatenating from " + str(min_offset*1.92) + " to " + str(max_offset*1.92) )
        tensor_to_wav(wav_id,inf_tensor , min_offset , max_offset)
        #print("now emptying the offset list...")
        offset_to_concat.clear()
        start_ind = next_ind
        next_ind = start_ind + 1
        #print("start_ind = " + str(start_ind))
        #print("next_ind = " + str(next_ind))
            
    

In [48]:
def tensor_to_wav(file_id,inf_tensor , min_offset , max_offset):
    """
   This function reads a tensor corresponding to the input audio and saves it as a wav file
   min_offset - starting index to read
   max_offset = end of index
    """
    
    out_dir = "../data/audio"
    #print("inside tensor_to_wav")
    #print("shape of inference tensor is - " + str(inf_tensor.shape))
    file_name =  out_dir +"/" +"child_" +str(file_id) + "_"+ str(int(min_offset*1.92)) +"_" +str(int(max_offset*1.92))   +".wav"
    #print("file_name = " + str(file_name))
    #print("min_offset = " +str(min_offset))
    #print("max_offset = " +str(max_offset))
    #print("inside tensor_to_wav .. input tensor shape = " + str(inf_tensor.shape))
    
    #reading the tensor
    stride = int(config.rate*config.min_duration)
    x_temp = inf_tensor[:,min_offset*stride:max_offset*stride]
    print("x_temp shape = " + str(x_temp.shape))
    torchaudio.save(file_name ,x_temp,sample_rate= config.rate )
    

In [ ]:
#Specify the id of the wav file 
#MOSQ SHORT = 127 
#MOSQ LONG = 199902
# BG SHORT = 222669
# BG LONG  = 222670
out_dir = "../outputs/labels_audacity"
uuid = '050aab6fc4651530'

wav_list = ["r2022-07-26_00.00.00.410__v6.aac_u2022-07-26_06.00.24.521642.aac"]
concat_successive = False
#file_name_dict = dict.fromkeys(wav_list)
print(file_name_dict)
for inp in wav_list:
    #inp = int(i)
    inf_file = "../data/audio/" + str(inp) 
    print(inf_file)
    df_y_hat = infer_model(model_MED, model_MSC ,inp , classes)
    df_y_hat.dropna(inplace=True)
    #df_y_hat.reset_index(inplace = True)
    #print(file_name_dict)
    write_file(df_y_hat, out_dir,i,classes)
    if concat_successive:
        concat_frames(df_y_hat,i,threshold = .5 ,every_mozz_frame = False , rate = config.rate)
        
        
        
    



{'r2022-07-26_00.00.00.410__v6.aac_u2022-07-26_06.00.24.521642.aac': None}
../data/audio/r2022-07-26_00.00.00.410__v6.aac_u2022-07-26_06.00.24.521642.aac
inside infer_model ....file = r2022-07-26_00.00.00.410__v6.aac_u2022-07-26_06.00.24.521642.aac
file_exists = True
number of inference windows = 1875
Frame = 0
 ***********Mosq Detected*********** 
mosq detected from 0.0 seconds  to 1.92 seconds 
Frame = 1
 ***********Mosq Detected*********** 
mosq detected from 1.92 seconds  to 3.84 seconds 
Frame = 2
 ***********Mosq Detected*********** 
mosq detected from 3.84 seconds  to 5.76 seconds 
Frame = 3
 ***********Mosq Detected*********** 
mosq detected from 5.76 seconds  to 7.68 seconds 
Frame = 4
 ***********Mosq Detected*********** 
mosq detected from 7.68 seconds  to 9.6 seconds 
Frame = 5
 NO mosq detected from 9.6 seconds  to 11.52 seconds 
Frame = 6
 ***********Mosq Detected*********** 
mosq detected from 11.52 seconds  to 13.44 seconds 
Frame = 7
 NO mosq detected from 13.44 second

In [ ]:
temp = ["child_199884_011"]
for i in temp:
    print(i)
    

m4a_file = '20211210_151013.m4a'
wav_filename = r"F:\20211210_151013.wav"
from pydub import AudioSegment
track = AudioSegment.from_file(m4a_file,  format= 'm4a')
file_handle = track.export(wav_filename, format='wav')

In [ ]:

def aac_to_wav(aac_file , file_name , rate = config.rate):
    path = "../data/audio/"
    wav_name = "../data/audio/" + file_name + ".wav"
    from os.path import exists
    file_exists = exists(aac_file)
    print("file_exists = " +str(file_exists))
    from pydub import AudioSegment
    track = AudioSegment.from_file(aac_file,  format= 'aac')
    file_handle = track.export(wav_name, format='wav')
    return file_handle

  
      
    
    

In [ ]:
#!pip install ffmpeg
#import ffmpeg
inp = "r2022-07-26_00.00.00.410__v6.aac_u2022-07-26_06.00.24.521642.aac"
split_tup = os.path.splitext(inp)
print(split_tup)
if split_tup[1] != ".wav" :
    print("not wav")
    inp_new = "../data/audio/"+ inp
    print("inp_new = " +str(inp_new))
    hand = aac_to_wav(inp_new , split_tup[0])
    #wav = "../data/audio/" + split_tup[0]+".wav"
  
    #print("wav = " + str(wav))
    inf_tensor,inp_samp_rate = torchaudio.load(hand)
    print(inf_tensor.shape)




## The Code below is to plot an inference output by each frame

In [ ]:
#converting to a wide form
dfm = df_y_hat.melt(id_vars='offset')


In [ ]:
dfm_new = dfm
print("dfm_new cols = " + str(dfm_new.columns))
dfm_new.rename(columns = {'offset':'Frame','variable':'Specie','value':'Probability'},inplace = True)
print("dfm_new cols = " + str(dfm_new.columns))

In [ ]:
#dfm_new

In [ ]:
# This function does the plot for each frame
def plot_specie_frame(dfm_new):
    
    g = sns.catplot( x = 'Specie',y = 'Probability', data=dfm_new , col = "Frame" , kind = "bar" , col_wrap= 5,sharex = True)
    g.set_xticklabels(rotation=90)
    #g.fig.set_size_inches(15, 8)
    g.fig.subplots_adjust(top=0.9)
    g.fig.suptitle('Mosq probability with Annotations')

    for ax in g.axes.ravel():
    # add annotations
        for c in ax.containers:
            labels = [round(v.get_height(),2)  for v in c]
            ax.bar_label(c, labels=labels)
        ax.margins(y=0.2)

    plt.show()

In [ ]:
plot_specie_frame(dfm_new)

In [ ]:
x_temp = torch.rand(1, 104000)
print(x_temp.shape)
stride = int(config.rate*config.min_duration)
print(9*stride)
print(10*stride)
x1 = x_temp[:,0*stride:6*stride]
print(x1.shape)
x2 = x_temp[:,9*stride:10*stride]
print(x2.shape)
9*stride- 10*stride

In [ ]:
nn.Softmax(.357 , .654)